# Monte Carlo Methods III: Parameter Estimation and Markov Chain

This hands-on is based on [Gravitational Wave Open Data Workshop 2024](https://github.com/gw-odw/odw-2024/).

## Introduction

Gravitational waves are ripples in spacetime predicted by Albert Einstein's General Theory of Relativity.
These waves propagate outward from sources such as merging black holes, neutron star collisions, or rapidly rotating neutron stars.
The first direct detection of gravitational waves, GW150914, was observed by the LIGO detectors in 2015, opening a new window into observing astrophysical phenomena previously inaccessible by electromagnetic observations alone.

Detecting and analyzing gravitational waves allows astronomers and physicists to study objects and phenomena that emit very little or no light, providing valuable insights into the behavior of gravity in extreme environments and the properties of dense objects like black holes and neutron stars.

In this hands-pon, we introduce Bayesian inference methods essential for estimating the parameters of gravitational wave signals detected by instruments such as LIGO and Virgo.
Specifically, we'll demonstrate parameter estimation using Monte Carlo Markov Chain.

## Bayesian Inference: A Brief Overview

Recalling that Bayesian inference allows us to update our knowledge of model parameters based on observed data.
Bayes' theorem mathematically represents this process:
\begin{align}
  p(\theta|d,M) = \frac{\mathcal{L}(d|\theta,M) \pi(\theta|M)}{\mathcal{Z}(d|M)}
\end{align}
where:
* $p(\theta|d, M)$ is the posterior distribution.
* $\mathcal{L}(d|\theta, M)$ is the likelihood function, measuring agreement between model predictions and observed data.
* $\pi(\theta|M)$ is the prior distribution, representing previous knowledge.
* $\mathcal{Z}(d|M)$ is the evidence, normalizing the posterior.

Typically, the posterior cannot be computed analytically, requiring computational approximations.
Stochastic (Monte Carlo) sampling is a common method for this purpose.

## Toy Model

We download sample observational data (`toy_model.csv`) containing simulated time-series observations:

In [ ]:
import os
if not os.path.isfile("toy_model.csv"):
    print("Downloading toy_model.csv")
    ! wget https://raw.githubusercontent.com/gw-odw/odw-2024/main/Tutorials/Day_3/toy_model.csv
else:
    print("toy_model.csv exists; not downloading")

Our example data (`toy_model.csv`) contains measurements (`yobs`) recorded at specific times (`time`).
Let's visualize these observations:

In [ ]:
import numpy as np

time, yobs = np.genfromtxt("toy_model.csv", delimiter=",").T

In [ ]:
from matplotlib import pyplot as plt

plt.plot(time, yobs)
plt.xlabel("Time [s]")
plt.ylabel("Observed values")
plt.title("Simulated Observational Data")

We propose the following sine-Gaussian model to explain the data:
\begin{align}
  s(t; f,\alpha) = e^{-(t/\alpha)^2} \sin(2\pi f t),
\end{align}
with frequency parameter $f$ and damping parameter $\alpha$.
To build intuition, we visualize this model for representative parameters:

In [ ]:
def sinusoid(time, freq):
    return np.sin(2 * np.pi * freq * time)

def gaussian_exponential(time, alpha):
    return np.exp(-(time/alpha)**2)

def model_Ms(time, freq, alpha):
    return gaussian_exponential(time, alpha) * sinusoid(time, freq)

In [ ]:
freq, alpha = 2, 0.5

plt.plot(time, sinusoid(time, freq), label="Sinusoid")
plt.plot(time, gaussian_exponential(time, alpha), label="Gaussian envelope")
plt.plot(time, model_Ms(time, freq, alpha), label="Sine-Gaussian")
plt.xlabel("Time [s]")
plt.ylabel("Model components")
plt.legend()